In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from mpl_toolkits.mplot3d import Axes3D
import random
import math
%matplotlib inline

In [26]:

# %matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (10, 10)

from google.colab import drive
drive.mount('/content/drive')

File="/content/drive/MyDrive/Assignment 1 Data/Regression/UnivariateData/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:

def readData(file):
    data=pd.read_csv(file+".txt",",",header=None)
    data=np.array(data.values)
    return data
    
Folder=[File+"32"]
def train_test_split(arr, test):
    y=np.full(len(arr),1)
    
    train= np.array([y[:-int(len(arr)*(test))],arr[:-int(len(arr)*(test)),0],arr[:-int(len(arr)*(test)),1]])
   
    test=np.array([y[-int(len(arr)*test): ],arr[-int(len(arr)*test): ,0],arr[-int(len(arr)*test): ,1]])
    
    return train.T, test.T

for k in Folder:
        
    folder=k
    data =  readData(k)
    train_data, Test=  train_test_split(data,0.20)




/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only


IndexError: ignored

In [ ]:



# Decision boundary
def generate_values_boundary2(data,weights,archicture,ttl):
   

    
    network_outputs=[]
    for i in range(len(data)):

        input_data=np.array(data[i][:2])
#         network_outputs.append(input_data) 
#         print(input_data)
        #Calculation of the weighted sum at the every layer of the neural network
        for h in range(len(archicture)-1):

            if h== len(archicture)-2:
                output_vector=np.dot(weights[h],input_data)
#                 print(output_vector)
            else:
                output_vector=perceptron(weights[h],input_data,sigmoid)
#                 print(output_vector)
            input_data=np.concatenate(([1],output_vector))
            
        network_outputs.append(output_vector[0])


    z= np.array(network_outputs)
#     print(data[:,3])  
#     print(z)
    plt.scatter(data[:,2],z,marker='D',color='#EAA358',edgecolors='black')
    
    
    
    print(ttl)
    plt.xlabel("Target Output")  # add X-axis label 
    plt.ylabel("Model Output") 
#     plt.savefig(ttl+'.png',bbox_inches = 'tight')
    plt.show()


In [ ]:

# Decision boundary
def generate_values_boundary(data,archicture,weights,sigmoid,ttl):
    min_x=min(data[:,1])
    max_x=max(data[:,1])
    x_mesh = np.linspace(min_x, max_x, 500)

    y_pred=[]
    for i in range(len(x_mesh)):
        input_data=np.copy(np.array([1,x_mesh[i]]))
        for h in range(len(archicture)-1):

            if h== len(archicture)-2:
                output_vector=np.dot(weights[h],input_data)
            else:
                output_vector=perceptron(weights[h],input_data,sigmoid)
                
            input_data=np.concatenate(([1],output_vector))
            
        y_pred.append(output_vector)
   
    
    plt.plot(x_mesh,y_pred,color='b',label='p1')
    
                    
    plt.scatter(data[:,1],data[:,2],marker='D',color='#EAA358',edgecolors='black')
    print(ttl)
    plt.xlabel("x1")  # add X-axis label 
    plt.ylabel("x2") 
    plt.savefig(ttl+'.png',bbox_inches = 'tight')
    plt.show()

plt.scatter(data[:,0],data[:,1],marker='D',color='#EAA358',edgecolors='black')
plt.xlabel("x1")  # add X-axis label 
plt.ylabel("x2") 
plt.savefig('data.png',bbox_inches = 'tight')

def logistic(input):
    x=np.array(input,dtype=np.float128)
    return 1/(1+np.exp(-x))

def perceptron(weights,input,sigmoid):
    
    predict=np.dot(weights,input)
    
    if(sigmoid==1):
        predict=np.tanh(predict)
        
    elif sigmoid==2:
        return predict
        
    else:
        predict=logistic(predict)
        
    return predict

def reg_derivative(x,sigmoid):
 
    return 1

def activation_derivative(x,sigmoid):
    
    if sigmoid:
        t=(np.exp(x)-np.exp(-x))/(np.exp(x)+np.exp(-x))
        dt=1-t**2
    
    else:
        dt= x * (1.0 - x)
        
    return dt

def calculate_delta(a,b):
    
    c=np.zeros(a.shape)
    for i in range(a.shape[0]):
        for j in range(a.shape[1]):

            if j!=0:
                c[i][j]=a[i][j]*b[i]
    
    return np.sum(c,axis=0)[1:]

def network(data,archicture,epoch,sigmoid,learning_rate):
    
    weights=[]

    for h in range(len(archicture)-1):
        weight=np.array([[random.seed(i+j) or random.random() for i in range(archicture[h]+1)] for j in range(archicture[h+1])])
        weights.append(weight)
        
    Error_epoch=list()
    
    
    for j in range(epoch):
        

        output=list()
        expected=list()
        np.random.shuffle(data)
        
        for i in range(len(data)):
            
            input_data=np.copy(data[i][:2])
            print(input_data)
        
            network_outputs=[]
            network_outputs.append(input_data) 
            
            #Calculation of the weighted sum at the every layer of the neural network
            for h in range(len(archicture)-1):
                
                if h== len(archicture)-2:
                    output_vector=np.dot(weights[h],input_data)
                else:
                    output_vector=perceptron(weights[h],input_data,sigmoid)
                    
                input_data=np.concatenate(([1],output_vector))
                network_outputs.append(input_data) 
#             print('output',input_data)
           
            #Send the data for backpropogation 
            target=np.copy(data[i][2])  
            weights=back_propagate(archicture,target,network_outputs,weights, learning_rate,sigmoid) 
            output.append(network_outputs[-1][1:])
            expected.append(list(data[i][2:]))
            
            
        #Calculate the MSE for the every epoch  
        error_output=np.array(expected)-np.array(output)
        cost=(np.sum(error_output*error_output,axis=0)/len(error_output))
        Etotal=np.sum(cost)
        Error_epoch.append(Etotal)
        
        #print the error of every output neuron in every 100 epochs
#         if j%100==0:
#             print('Epoch',j,': ',cost)

        
    
    return Error_epoch,weights


In [ ]:

def back_propagate(archicture,target,network_outputs,weights, learning_rate,sigmoid):
    
    Delta=[]
    length=len(archicture)
    
    #Calculate the delta for the final output layer
    Do=-2*(target-network_outputs[-1:][0][1:])*reg_derivative(1,sigmoid)
    Delta.append(Do)
    
    #Claculate the delta for all internal layers
    for h in reversed (range(1,length-1)):
        new_Delta=calculate_delta(weights[h],Delta[-1])
        new_Delta=new_Delta*activation_derivative(network_outputs[h-length][1:],sigmoid)
        Delta.append(new_Delta)
    Delta.reverse()
    
    #Calculate the new updated value for the weights   
    
    k=-1
    for W in weights:
        k=k+1
        for i in range(W.shape[0]):
            for j in range(W.shape[1]):
                W[i][j]=W[i][j]-learning_rate*network_outputs[k][j]*Delta[k][i]
        
    return weights


In [ ]:

def classifier(data,archicture,epoch,sigmoid,learning_rate,file):
    
    error_output,weights=network(np.copy(data),archicture,epoch,sigmoid,learning_rate)
    
    plt.plot(error_output)
    plt.xlabel("Epochs")  # add X-axis label 
    plt.ylabel("Error")  # add Y-axis label 
    plt.title("Error Plot")  # add title 
    #plt.savefig(file+'Figure/Uni_Error.png',bbox_inches = 'tight')


    
    return weights

def test1(data,weights,archicture,sigmoid):
    

        output=list()
        neu_output=list()
        expected=list()
        np.random.shuffle(data)
        
        for i in range(len(data)):
            
            input_data=np.copy(data[i][:2])
            print(input_data)
        
            network_outputs=[]
            network_outputs.append(input_data) 
            
            #Calculation of the weighted sum at the every layer of the neural network
            for h in range(len(archicture)-1):
                
                if h== len(archicture)-2:
                    output_vector=np.dot(weights[h],input_data)
                else:
                    output_vector=perceptron(weights[h],input_data,sigmoid)
                    neu_output.append(output_vector)
                    
                input_data=np.concatenate(([1],output_vector))
                network_outputs.append(input_data) 
#             print('output',input_data)
           
            #Send the data for backpropogation 
            target=np.copy(data[i][2])  
            back_propagate(archicture,target,network_outputs,weights, learning_rate,sigmoid)
            output.append(network_outputs[-1][1:])
            expected.append(list(data[i][3:]))
            
            
        #Calculate the MSE for the every epoch  
        error_output=np.array(expected)-np.array(output)
        cost=(np.sum(error_output*error_output,axis=0)/len(error_output))
        Etotal=np.sum(cost)
        
       

        
    
        return Etotal,neu_output

def cross_validation(k,Arr):



    length1=int(len(Arr)*0.25)

    
    M1,M2,M3,M4=Arr[:length1].copy(),Arr[length1:2*length1],Arr[2*length1:3*length1],Arr[3*length1:4*length1]
   
    if k==1:
        M=np.concatenate((M2,M3,M4))
        V=M1
    if k==2:
        M=np.concatenate((M1,M3,M4))
        V=M2
    if k==3:
        M=np.concatenate((M1,M2,M4))
        V=M3
    if k==4:
        M=np.concatenate((M1,M2,M3))
        V=M4


    
    
    return M, V

#archicture[no_of_input,no_of_neuron_h1,no_of_neuron_h2,no_of_output]
#archicture[no_of_input,no_of_neuron_h1,no_of_output]
#sigmoid=0 means logistic and sigmoid=1 means tanh

archicture=[1,3,1]
sigmoid=0
learning_rate=0.1
epoch=50
k=3 # k=1, 2, 3, 4 for the cross validation, where 1 is Fold-1
plt.rcParams['figure.figsize'] = (10, 10)



Train,Validation=cross_validation(k,train_data)
Weights=classifier(train_data,archicture,epoch,sigmoid,learning_rate,File)

generate_values_boundary(Train,archicture,Weights,sigmoid,'Training Data')

def test(data,weights,archicture,sigmoid):
    
        output=[]
        expected=[]
        neu_output=list()
        for i in range(len(data)):
            
            input_data=np.copy(data[i][:2])
        
            network_outputs=[]
            network_outputs.append(input_data) 
            
            #Calculation of the weighted sum at the every layer of the neural network
            for h in range(len(archicture)-1):
                
                if h== len(archicture)-2:
                    output_vector=np.dot(weights[h],input_data)
#                     neu_output.append(output_vector)
                else:
                    output_vector=perceptron(weights[h],input_data,sigmoid)
                    neu_output.append(output_vector)
                    
                input_data=np.concatenate(([1],output_vector))
                network_outputs.append(input_data) 
#             print('output',input_data)
           
            #Send the data for backpropogation 
            target=np.copy(data[i][2])  
            output.append(network_outputs[-1][1:])
            expected.append(list(data[i][2:]))
            
            
        #Calculate the MSE for the every epoch  
        error_output=np.array(expected)-np.array(output)
        cost=(np.sum(error_output*error_output,axis=0)/len(error_output))
        Etotal=np.sum(cost)
        

        
    
        return Etotal,neu_output

a=[0.0031,0.0034,0.0036,0.0042]
b=[0.0042,0.0035,0.0029,0.0038]
c=[0.0036,0.0036,0.0036,0.0040]
xticks = ['Model 1','Model 2','Model 3','Model 4']

plt.plot(xticks,a,label='Train Error')
plt.plot(xticks,b,label='Validation Error')
plt.plot(xticks,c,label='Test Error')
plt.ylabel("MSE")  # add X-axis label 
plt.xlabel("Model Complexity") 
plt.legend()

A1,output1=test(Train,Weights,archicture,sigmoid)
A2,output2=test(Validation,Weights,archicture,sigmoid)
A3,output3=test(Test,Weights,archicture,sigmoid)
print('Training Error: ',A1)
print('Validation Error: ',A2)
print('Test Error: ',A3)

def plot(data,output):
    
    plt.rcParams['figure.figsize'] = (20, 30)
    # plt.rcParams['axes.facecolor']='w'
    fig = plt.figure()
    ax1 = fig.add_subplot(421)
    ax2 = fig.add_subplot(422)
    ax3 = fig.add_subplot(423)
#     ax4 = fig.add_subplot(424, projection='3d')
#     ax5 = fig.add_subplot(425, projection='3d')
#     ax6 = fig.add_subplot(426, projection='3d')
#     ax7 = fig.add_subplot(427, projection='3d')
#     ax8 = fig.add_subplot(428, projection='3d')

    axs=[ax1,ax2,ax3]
    

    for j in range(len(axs)):
        axs[j].set_xlabel('X Label')
        axs[j].set_ylabel('Y Label')
        # axs[j].set_zlabel('Z Label') 

    for j in range(len(axs)):

        for i in range (len(data)):

            # Data for three-dimensional scattered points
            r=j+1
            l=r-1

            zdata = output[i][l:r]
            xdata = data[i][1:2]
            ydata = data[i][2:3]
            axs[j].scatter(xdata, zdata, color='r',edgecolors='black');


    # plt.savefig('Figure/Neuron_output.png',bbox_inches = 'tight')

plot(Train,output1)

plot(Validation,output2)

plot(Test,output3)
def plot2(data,output):
    
    plt.rcParams['figure.figsize'] = (10, 10)
    # plt.rcParams['axes.facecolor']='w'
    fig = plt.figure()
    ax1 = fig.add_subplot(111)


    axs=[ax1]

    for j in range(len(axs)):
        axs[j].set_xlabel('X Label')
        axs[j].set_ylabel('Y Label')
        # axs[j].set_zlabel('Z Label') 

    for j in range(len(axs)):

        for i in range (len(data)):

            # Data for three-dimensional scattered points
            r=j+1
            l=r-1

            zdata = output[i][l:r]
            xdata = data[i][1:2]
            ydata = data[i][2:3]
            axs[j].scatter(xdata, zdata, color='r',edgecolors='black');


    # plt.savefig('Figure/Neuron_output.png',bbox_inches = 'tight')


In [ ]:

# Decision boundary
def generate_values_boundary2(data,weights,archicture,ttl):
   

    
    network_outputs=[]
    for i in range(len(data)):

        input_data=np.array(data[i][:2])
#         network_outputs.append(input_data) 
#         print(input_data)
        #Calculation of the weighted sum at the every layer of the neural network
        for h in range(len(archicture)-1):

            if h== len(archicture)-2:
                output_vector=np.dot(weights[h],input_data)
#                 print(output_vector)
            else:
                output_vector=perceptron(weights[h],input_data,sigmoid)
#                 print(output_vector)
            input_data=np.concatenate(([1],output_vector))
            
        network_outputs.append(output_vector[0])


    z= np.array(network_outputs)
#     print(data[:,3])  
#     print(z)
    plt.scatter(data[:,2],z,marker='D',color='#EAA358',edgecolors='black')
    
    
    
    print(ttl)
    plt.xlabel("Target Output")  # add X-axis label 
    plt.ylabel("Model Output") 
#     plt.savefig(ttl+'.png',bbox_inches = 'tight')
    fig = plt.figure(figsize=(8,6))
    plt.show()

generate_values_boundary2(Train,Weights,archicture,"r")

generate_values_boundary2(Validation,Weights,archicture,"r")
generate_values_boundary2(Test,Weights,archicture,"r")